In [2]:
import requests
from bs4 import BeautifulSoup as bs
from selenium import webdriver
from selenium.webdriver.common.by import By
import pandas as pd
import time
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
# 사용에 아무런 의미가 없음 걍 쓴거임.
# import undetected_chromedriver as uc

# 대기 시간 1초
LOADING_WAIT_TIME= 1
# 크롤링할 상품 코드
pcodes = ['14142050', '13150841', '13924424']
# 결과 리스트
result = []


def init_driver():
    driver = webdriver.Chrome('diver/chromedriver.exe')
    return driver


def find_review(pcode, driver):
    url = f'http://prod.danawa.com/info/?pcode={pcode}'
    driver.get(url)
    # 제휴 쇼핑몰 리뷰 클릭
    # 페이지 로딩시 , 로딩시만큼 기다린다
    # 이 떄 EC.presence_of_element_located에서 해당 엘리먼트가 나올 때까지 기다린다.
    WebDriverWait(driver, LOADING_WAIT_TIME).until(
        EC.presence_of_element_located(
            (By.XPATH,'//*[@id="danawa-prodBlog-productOpinion-button-tab-companyReview"]')
        )
    )
    driver.find_element(By.XPATH,'//*[@id="danawa-prodBlog-productOpinion-button-tab-companyReview"]').click()

    for i in range(1, 1000):
        # 이번 페이지의 모든 리뷰가 로드 될때까지 기다림
        WebDriverWait(driver, LOADING_WAIT_TIME).until(
            EC.visibility_of_all_elements_located(
                (By.CLASS_NAME,'atc')
            )
        )
        for review in driver.find_elements(By.CLASS_NAME,'atc'):
            result.append(review.text)

        
        if i % 10 == 0:
            # 10개를 다 봤다면 다음 10개를 보는 버튼 클릭, 만약 클릭이 안되면 종료
            right_btn = driver.find_element(By.XPATH, '//*[@class="mall_review"]//span[@class="point_arw_r"]')
            if right_btn.value_of_css_property('cursor') == 'pointer':
                right_btn.click()
            else:
                break
        else:
            try: # 다음 페이지가 없을 경우
                WebDriverWait(driver, LOADING_WAIT_TIME).until(
                        EC.presence_of_element_located(
                            (By.XPATH, f'//*[@id="danawa-prodBlog-companyReview-content-list"]/div/div/div/a[text()={i + 1}]')
                        )
                    )
            except:
                break
            driver.find_element(By.XPATH, f'//*[@id="danawa-prodBlog-companyReview-content-list"]/div/div/div/a[text()={i + 1}]').click()
if __name__ == "__main__":
    driver = init_driver()
    for pcode in pcodes:
        find_review(pcode, driver)
    print(len(result))
    for r in result:
        print(r)

C:\Users\PiGiraffe0\AppData\Local\Temp\ipykernel_25320\406681113.py:22: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('C:/Users/PiGiraffe0/Documents/multicampus_final_First/final_project/참고자료/review_crawling/diver/chromedriver.exe')


StaleElementReferenceException: Message: stale element reference: element is not attached to the page document
  (Session info: chrome=108.0.5359.125)
Stacktrace:
Backtrace:
	(No symbol) [0x00B5F243]
	(No symbol) [0x00AE7FD1]
	(No symbol) [0x009DD04D]
	(No symbol) [0x009DFD34]
	(No symbol) [0x009DFBE5]
	(No symbol) [0x009DFE80]
	(No symbol) [0x00A07D27]
	(No symbol) [0x00A2858C]
	(No symbol) [0x00A02BFF]
	(No symbol) [0x00A28804]
	(No symbol) [0x00A3C9EB]
	(No symbol) [0x00A28386]
	(No symbol) [0x00A0163C]
	(No symbol) [0x00A0269D]
	GetHandleVerifier [0x00DF9A22+2655074]
	GetHandleVerifier [0x00DECA24+2601828]
	GetHandleVerifier [0x00C08C0A+619850]
	GetHandleVerifier [0x00C07830+614768]
	(No symbol) [0x00AF05FC]
	(No symbol) [0x00AF5968]
	(No symbol) [0x00AF5A55]
	(No symbol) [0x00B0051B]
	BaseThreadInitThunk [0x774C00F9+25]
	RtlGetAppContainerNamedObjectPath [0x77877BBE+286]
	RtlGetAppContainerNamedObjectPath [0x77877B8E+238]


In [5]:
result

['가격: 우주패스 덕에 2만원 후반대에 구매했습니다.배송: 2틀 걸림사용 후기-에어태그 ？ 떼자마자 바로 핸드폰 연동되어서 설정 간편했습니다!!! 물건 이름 설정해주면 됩니다.이후에 나의 찾기 들어가서 소리 켜보니 알림 잘 되네요사이즈는 생각보다 작아요;;;; 500원보다 좀 큰 듯ㅎ그래도 잘 사용할 것 같습니다저는 에어팟 본체에 걸었어요',
 '한국에서는 기능을 모두 사용할 수 없는 점이 아쉽지만 해외에서 사용할 목적으로 키링과 함께 주문했는데 키링은 아직 도착하지 않았네요.',
 '키링에 꼽아서 유용하게 사용하고 있어요. 어디에 있던지 찾을수 있어요. 빠른배송과 합리적인 가격도 마음에 들어요',
 '7% 브랜드 할인이 있고 차도 새로 받기로 하는게 있어서 구매했습니다. ？x면은 바둑돌 같고 역시 깔끔하네요.',
 '좋어요~리모컨 집안에서 쓰다가 자주 못찾는 편이여서 구매햇어요~배송도 빠르고 디자인 이쁘고 잘됩니다!',
 '아직 한국내에서는 위치 추적이 안되서 아쉽네요.배송은 이틀이나 걸렸네요. 슈팅배송이라고 했는데..ㅠㅠ',
 '궁금해서 구입했어요! 크게 한국에서는 쓸모는 없긴 하지만 그래도 물건 어디에 두었는지 방향 정도는 나오니 재밌어요',
 '빠른 배송은 만족합니다만 22년 5월에 21년 5월에 제조된 상품을 발송하는 게 말이 되나요?',
 '빠른 배송은 물론이고 제품의 포장 및 빠른 제품 인식까지 굿!11절에 저렴하게 잘 샀습니다.',
 '차키 찾을때마다 짜증낫는데 해결됨자주 쓰고 가끔 어디두엇는지 모르는 급하게 찾는물건찾는데 좋아요',
 '에어팟을 매번 어디 두는지 깜빡하는데 넘 편합니다!!!집안에서 찾기 넘 좋아요!!! 강추',
 '열쇠고리에 걸려고 주문했어요. 집안에 가끔 어디다 두었는지 햇갈려서 찾는용도로 구매함.',
 '다음날 배송이라고 해서 주문했는데 이틀 걸렸어요. 급해서 주문한건데. 슈팅배송 아니에요',
 '배송은 그렇고 실제 호환이나 설정은 쉽게 했네요 워낙 간단한 거라 잘 쓰고 있어요',
 '호기심에 구매 했는데 만족 합니다. 배